In [1]:
import pandas as pd
import calendar
import datetime
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

## real price data cleaning

In [2]:
df_real_price = pd.read_csv(r'D:\Shuai-Jingbo-Pei-yu\CoinMarketCap_CoinPrice_data\Coin_real_price\merged\2021-2022.csv')

In [3]:
def get_real_price(coin_symbol,start_date = None,end_date = None):
    df_real_price_cleaned = df_real_price.drop(df_real_price[df_real_price.loc[:,'Date'].str.split('-',expand= True)[1].isna()].index)
    df_real_price_cleaned.loc[:,'Date'] = pd.to_datetime(df_real_price_cleaned.loc[:,'Date'],format = "%d-%b-%y")
    df_btc_daily = df_real_price_cleaned[df_real_price_cleaned.loc[:,'Symbol'] == coin_symbol]
    if end_date == None:
        df_return = df_btc_daily[df_btc_daily.loc[:,'Date'] >= start_date]
    elif start_date == None:
        df_return = df_btc_daily[df_btc_daily.loc[:,'Date'] <= end_date]
    else:
        df_return = df_btc_daily[(df_btc_daily.loc[:,'Date'] >= start_date) & (df_btc_daily.loc[:,'Date'] <= end_date)]
    df_return.loc[:,'Price'] = pd.to_numeric(df_return.loc[:,'Price'])
    return df_return
# get_real_price('BTC',start_date = '2022-01-01')

In [4]:
# fig = px.line(df_btc,x = 'Date',y ='Price',text = 'Price',title = "Price of BTC from Jan 2021 to May 2022")
# fig.update_traces(textposition="top right")


## predication data cleaning

In [5]:
df_predication_before_jun = pd.read_csv(r'D:\Shuai-Jingbo-Pei-yu\CoinMarketCap_CoinPrice_data\Daily Prediction\src\to_one_&_to_week\total-before-jun.csv')
df_predication_after_jun = pd.read_csv(r'D:\Shuai-Jingbo-Pei-yu\CoinMarketCap_CoinPrice_data\Daily Prediction\src\to_one_&_to_week\total-after-jun.csv')
df_predication_total = pd.concat([df_predication_before_jun,df_predication_after_jun])

In [66]:
# get_predication('BTC','2022-06-30')

In [61]:
def get_predication(symbol,predication_date):
    df_predication_total_date_formated = df_predication_total.copy()
    df_predication_total_date_formated.loc[:,'Predication_month_year'] = pd.to_datetime(df_predication_total.loc[:,'Predication_month_year'],format = '%b-%y') + pd.offsets.MonthEnd()
    df_predication_total_date_formated.loc[:,'Date'] = pd.to_datetime(df_predication_total_date_formated.loc[:,'Date'],format = '%d-%b-%y')
    df_predication_symbol = df_predication_total_date_formated[df_predication_total_date_formated.loc[:,'Crypto_symbol']==symbol].sort_values(by = ['Date','Predication_month_year'])
#     df_predication_symbol = df_predication_symbol[df_predication_symbol.loc[:,'Predication_month_year']]
    df_real_price = get_real_price(symbol,start_date = '2021-11-01')
    df_symbol_price = df_real_price.loc[:,['Price','Date']]
#     display(df_symbol_price)
#     display(df_predication_symbol)
    df_merged = df_predication_symbol.merge(df_symbol_price,
                             left_on = 'Predication_month_year',
                             right_on ='Date').rename(columns = {'Date_x':'Predication_date',
                                                                 'Date_y':'Price_date'})
    df_merged_month = df_merged[df_merged.loc[:,'Price_date'] == predication_date]
#     df_merged_month.loc[:,'Difference'] = df_merged_month.loc[:,'Values_median']-df_merged.loc[:,'Price']
#     df_merged_month.loc[:,'Percentage_Difference'] = df_merged_month.loc[:,'Difference']/df_merged_month.loc[:,'Price']
#     df_merged_month.sort_values(by = 'Predication_date',inplace = True)
    return df_merged_month

In [16]:
symbol_list = df_predication_total.iloc[-600:,:].drop_duplicates(subset = ['Crypto_symbol']).loc[:,'Crypto_symbol'].to_list()
symbol_list[:5]

['BTC', 'ETH', 'BNB', 'SOL', 'ADA']

## visualization

In [20]:
def visualize(symbol,month):
#     symbol = 'ETH'
#     month = '2022-03-31'
    df_merged = get_predication(symbol,month)
    df_btc_daily = get_real_price(symbol,'2021-11-01',month)
#     display(df_merged)
#     display(df_btc_daily)
    # visualization
    fig = go.Figure()
    # predication
    fig.add_trace(go.Scatter(x = df_merged['Predication_date'], y = df_merged['Values_median'],
                        name='Predication'))
    # real price
    text_list =  df_btc_daily['Price'].copy()
    text_list[:-1] = None
    fig.add_trace(go.Scatter(x = df_btc_daily['Date'], y = df_btc_daily['Price'],
                        name='real price',
                        text = text_list,
                        textposition = 'bottom center'
                            ))
    fig.update_layout(title = 'Predication to '+month+' and real price of '+symbol,
                     xaxis_title = 'Date',
                     yaxis_title = 'Price')
    return fig

In [ ]:
# fig_btc = visualize('BNB','2022-03-31')
# fig_btc.show()
# fig_eth = visualize('ETH','2022-06-30')
# fig_eth.show()
# for i in range(len(symbol_list)):
#     print('Rank: '+str(i))
#     fig = visualize(symbol_list[i],'2022-03-31')
#     fig.show()
#     if i > 20:
#         break
# df_merged = get_predication('BTC',"2022-03-31")
# df_btc_daily = get_real_price('BTC','2021-11-01',"2022-03-31")
# df_merged

In [89]:
def dropdown(month):
    buttons = []
    fig = go.Figure()
    df_merged = get_predication('BTC',month)
    df_btc_daily = get_real_price('BTC','2021-11-01',month)
    fig.add_trace(go.Scatter(x = df_merged['Predication_date'], y = df_merged['Values_median'],
                            name='Predication'))
    fig.add_trace(go.Scatter(x = df_btc_daily['Date'], y = df_btc_daily['Price'],
                            name = 'Real price'))
    for symbol in symbol_list[:30]:
        for month in [month]:
            df_merged = get_predication(symbol,month)
            df_btc_daily = get_real_price(symbol,'2021-11-01',month)
            buttons.append(dict(method='restyle',
                            label=symbol,
                            visible=True,
                            args=[{'y':[df_merged['Values_median'],df_btc_daily['Price']],
                                    'x':[df_merged['Predication_date'],df_btc_daily['Date']],
                                    'type':'scatter',
                                    'showlegend':True
                                  },
                                 ]))

    updatemenu = []
    your_menu = dict()
    updatemenu.append(your_menu)
    updatemenu[0]['buttons'] = buttons
    updatemenu[0]['direction'] = 'down'
    updatemenu[0]['showactive'] = True
    fig.update_layout(showlegend=False, updatemenus=updatemenu,xaxis_title='Date', yaxis_title='Price')
    fig.show()
    fig.write_html(r'D:\Shuai-Jingbo-Pei-yu\CoinMarketCap_CoinPrice_data\Daily Prediction\src\Data analysis'+'\\predication to '
                   +str(month)+'.html')

In [90]:
display('predication to 2022-03-31')    
dropdown('2022-03-31')
display('predication to 2022-04-30')
dropdown('2022-04-30')
display('predication to 2022-05-31')
dropdown('2022-05-31')


'predication to 2022-03-31'

'predication to 2022-04-30'

'predication to 2022-05-31'